In [1]:
from carball import analyze_replay_file
from carball.analysis.analysis_manager import AnalysisManager
import logging

def parse(replay_path: str) -> AnalysisManager:
    return analyze_replay_file(replay_path, logging_level=logging.FATAL)

In [2]:
import os
from carball.generated.api.metadata.game_metadata_pb2 import Playlist
from carball.generated.api.metadata import game_metadata_pb2

from DataCleaning import formatFrames

replay_directory = os.path.join(os.getcwd(), "replays")

valid_playlists = [
    game_metadata_pb2.UNRANKED_DUELS,
    game_metadata_pb2.UNRANKED_DOUBLES,
    game_metadata_pb2.UNRANKED_STANDARD,
    game_metadata_pb2.CUSTOM_LOBBY,
    game_metadata_pb2.RANKED_DUELS,
    game_metadata_pb2.RANKED_DOUBLES,
    game_metadata_pb2.RANKED_SOLO_STANDARD,
    game_metadata_pb2.RANKED_STANDARD,
    game_metadata_pb2.TOURNAMENT,
]

n_duels_columns = 14 * 2 + 10
n_doubles_columns = 14 * 4 + 10
n_standard_columns = 14 * 6 + 10

working_replays = 0
failing_replays = 0
invalid_columns = 0

working_duels = 0
working_doubles = 0
working_standard = 0

am: AnalysisManager

In [ ]:
for replay in os.listdir(replay_directory):
    if replay.endswith(".replay"):
        try:
            am = parse(os.path.join(replay_directory, replay))

            if am.protobuf_game.game_metadata.playlist not in valid_playlists:
                raise Exception(f"Invalid playlist: {Playlist.Name(am.protobuf_game.game_metadata.playlist)}\nID: {am.protobuf_game.game_metadata.playlist}")
            
            if am.protobuf_game.mutators.game_mutator_index != 0 and am.protobuf_game.mutators.game_mutator_index != -1:
                raise Exception(f"Invalid mutator index: {am.protobuf_game.mutators.game_mutator_index}")
            
            try:
                frames = formatFrames(am)
            except Exception as e:
                raise Exception(f"Failed to format frames: {e}")
            
            if len(frames.columns) == n_duels_columns:
                working_duels += 1
            elif len(frames.columns) == n_doubles_columns:
                working_doubles += 1
            elif len(frames.columns) == n_standard_columns:
                working_standard += 1
            else:
                invalid_columns += 1
                raise Exception(f"Invalid number of columns: {len(frames.columns)}")
            
            working_replays += 1
            
            print(f"Successfully parsed {replay}")
        except Exception as e:
            failing_replays += 1
            print(f"Failed to parse {replay}")
            print(e)
